In this file, I use Snowpark to load the full insurance dataset csv(1M rows) into a snowflake table 

In [3]:
from snowflake.snowpark import Session
import json
import pandas as pd

# Initiate session
connection_parameters = json.load(open('./connection.json'))
session = Session.builder.configs(connection_parameters).create()
snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
session.use_database('Insurance')
session.use_schema('ML_PIPE')

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))


Connection Established with the following parameters:
User                        : SPORTY4992
Role                        : "ACCOUNTADMIN"
Database                    : "INSURANCE"
Schema                      : "ML_PIPE"
Warehouse                   : "COMPUTE_WH"


In [4]:
!kaggle datasets download -d sridharstreaks/insurance-data-for-machine-learning --unzip

Dataset URL: https://www.kaggle.com/datasets/sridharstreaks/insurance-data-for-machine-learning
License(s): CC0-1.0
 94%|███████████████████████████████████▊  | 20.0M/21.3M [00:01<00:00, 11.7MB/s]
100%|██████████████████████████████████████| 21.3M/21.3M [00:01<00:00, 11.7MB/s]


In [5]:
# Load full 1M dataset into dataframe
insurance_df = pd.read_csv('insurance_dataset.csv')

Data cleaning, rearranging columns

In [ ]:
# Capitalize column names
insurance_df.columns = insurance_df.columns.str.upper()

# Rearrange columns to fit target schema
cols = insurance_df.columns.tolist()
cols = cols[:3] + cols[-1:] + cols[3:-1]
insurance_df = insurance_df[cols]

Use the write_pandas() method to write the first 10k rows into the 'SOURCE_OF_TRUTH' table created with the SQL commands in the SQL file. The method "returns a Snowpark DataFrame object referring to the table where the pandas DataFrame was written to." (Snowpark Documentation)

In [6]:
source_of_truth_df = session.write_pandas(insurance_df[:10000], table_name='SOURCE_OF_TRUTH',database='INSURANCE',schema='ML_PIPE',auto_create_table=True)

The code below writes the remaining 990k to the INCOMING_DATA_SOURCE table to simulate data being streamed in

In [7]:
incoming_data_source_df = session.write_pandas(insurance_df[10000:], table_name='INCOMING_DATA_SOURCE',database='INSURANCE',schema='ML_PIPE',auto_create_table=True)

/Users/Tom/anaconda3/envs/sf-ml-pipeline/lib/python3.11/site-packages/snowflake/snowpark/session.py:2940: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.range.RangeIndex'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, _, _, ci_output = write_pandas(
